# 学会使用Fiddler

# 爬虫浏览器的伪装技术

In [ ]:
import urllib.request
import http.cookiejar
url = "https://news.163.com/18/1221/22/E3J6A92F0001875P.html"
cjar = http.cookiejar.CookieJar()
proxy = urllib.request.ProxyHandler({'http':"127.0.0.1:8888"})
opener = urllib.request.build_opener(proxy,urllib.request.HTTPHandler,urllib.request.HTTPCookieProcessor(cjar))
urllib.request.install_opener(opener)
data = urllib.request.urlopen(url).read()
fhandle = open("d:/python/8_1.html","wb")
fhandle.write(data)
fhandle.close()

**注意：**如果使用fiddler作为代理服务器，所爬取的网址要以具体文件或者“/”结尾，如果有具体文件，直接写改文件的具体网址即可，“https://news.163.com/18/1221/22/E3J6A92F0001875P.html”这种写法是合法的。  
如果被爬取的是一个文件夹，比如”http://www.baidu.com"，此时爬取的是一个目录（文件夹），所以要以“/”结尾，“http://www.baidu.com"这种是合法的，而”http://www.baidu.com"这种则可能会出现错误

referer字段的值一般设为要爬取的网页的域名地址或对应的网站的主页网址。  
Accept-Encoding设置为gzip,deflate,可能会出现乱码，此时只需要将该字段信息省略不写或者将该字段信息设置为utf-8或gb2312即可。因为如果设置为gzip,deflate，从服务器返回的这两种压缩的代码，没有浏览器解码，会出现乱码问题。

## 小结

1）常见反爬虫机制：通过分析用户请求的Headers信息进行反爬虫，通过检测用户行为进行反爬虫，比如通过判断同一个IP在短时间内是否频繁访问对应网站等进行分析；通过动态页面增加爬虫爬取的难度，达到反爬虫的目的。  
2）使用fiddler作为代理服务器，所爬取的网址要以具体文件或者“/”结尾。  
3）referer字段的值一般可以设置为要爬取的网页的域名地址或对应网站的主页网址。  
4）在实际项目中，要伪装成浏览器，我们不一定要将fiddler设置为代理服务器，上面实例1、2设置Fiddler为代理服务器主要为了方便抓包，从而方便调试。

# 爬虫的定向爬取技术

## 什么是定向爬取技术

## 定向爬取的相关步骤与策略

定向爬取技术主要需要解决3个问题：  
1）清晰地定义好爬虫的爬取目标，规划好主题。  
2）建立好爬取网址的过滤筛选规则以及内容的过滤筛选规则。  
3）建立好URL排序算法，让爬虫能够明确优先爬取哪些网页、以什么顺序爬取待爬取的网页。不一样的顺序去爬取，可能会导致不一样的爬取效率。

## 定向爬虫实战

观察点击“加载更多评论”前后的请求网址变化，总结视频评论的URL地址格式为："https://video.coral.qq.com/article/视频编号/"
"https://video.coral.qq.com/varticle/1005669305/comment/v2?callback=_varticle1005669305commentv2&orinum=10&oriorder=o&pageflag=1&cursor=0&scorecursor=0&orirepnum=2&reporder=o&reppageflag=1&source=9&_=1545445817681"(起始页)
"https://video.coral.qq.com/varticle/1005669305/comment/v2?callback=_varticle1005669305commentv2&orinum=10&oriorder=o&pageflag=1&cursor=6399626802072370905&scorecursor=0&orirepnum=2&reporder=o&reppageflag=1&source=9&_=1545445817684"(加载一次)
"https://video.coral.qq.com/varticle/1005669305/comment/v2?callback=_varticle1005669305commentv2&orinum=10&oriorder=o&pageflag=1&cursor=6464381694372580731&scorecursor=0&orirepnum=2&reporder=o&reppageflag=1&source=9&_=1545445817685"(加载两次)  
点击网址查看，数据信息，发现评论id，被评论id，评论内容格式。

In [ ]:
import urllib.request
import http.cookiejar
import re
# 设置视频编号
# 大话西游之大圣娶亲https://v.qq.com/x/cover/fse52rd4klx7qn2.html?
vid = "1005669305"
# 设置评论起始标编号
cursorid = "0"
# 构造真实评论请求网址
url = "https://video.coral.qq.com/varticle/"+vid+"/comment/v2?callback=_varticle1005669305commentv2&orinum=10&oriorder=o&pageflag=1&cursor="+cursorid+"&scorecursor=0&orirepnum=2&reporder=o&reppageflag=1&source=9&_=1545445817681"
# 设置头信息伪装成浏览器
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gb2312, utf-8",
    "Accept-Language": "zh-CN,zh;q=0.9",
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36",
    "Host": "video.coral.qq.com",
}
#设置cookie
cjar = http.cookiejar.CookieJar()
opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cjar))
headall=[]
for key,value in headers.items():
    item = (key,value)
    headall.append(item)
opener.addheaders = headall
urllib.request.install_opener(opener)
#爬取该网页
data = urllib.request.urlopen(url).read().decode('utf-8')
#分别构建筛选id、用户名、评论内容等信息的正则表达式
idpat = '"targetid":"(.*?)"'#书上是“id”：
userpat = '"userid":"(.*?)",'#书上是“nick"
conpat = '"content":"(.*?)",'
#分别根据正则表达式查找所有id、用户名、评论内容等信息
idlist = re.compile(idpat,re.S).findall(data)
print('1')
userlist = re.compile(userpat,re.S).findall(data)
print(eval('u"'+userlist[1]+'"'))
conlist = re.compile(conpat,re.S).findall(data)
print(len(userlist))
#通过循环将获取到的各信息遍历出来
for i in range(0,10):
    #输出对应信息，并对字符串进行unicode编码，正常显示
    try:
        print("用户名是："+eval('u"'+userlist[i]+'"'))
    except Exception as e:
        print(str(e))
    print("评论内容是："+eval('u"'+conlist[i]+'"'))
    print("\n")

但是上方只能获取一页的评论，修改之后，获取多页：

In [18]:
import urllib.request
import http.cookiejar
import re
# 设置视频编号
# 大话西游之大圣娶亲
vid = "1005669305"
# 刚开始评论ID
cursorid = "0"
# 构造真实评论请求网址
url = "https://video.coral.qq.com/varticle/"+vid+"/comment/v2?callback=_varticle1005669305commentv2&orinum=10&oriorder=o&pageflag=1&cursor="+cursorid+"&scorecursor=0&orirepnum=2&reporder=o&reppageflag=1&source=9&_=1545445817681"
# 设置头信息伪装成浏览器
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gb2312, utf-8",
    "Accept-Language": "zh-CN,zh;q=0.9",
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36",
    "Host": "video.coral.qq.com",
}
#设置cookie
cjar = http.cookiejar.CookieJar()
opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cjar))
headall=[]
for key,value in headers.items():
    item = (key,value)
    headall.append(item)
opener.addheaders = headall
urllib.request.install_opener(opener)
#建立一个自定义函数craw(vid,comid),实现自动爬取对应评论网页并返回爬取数据
def craw(vid,comid):
    url = "https://video.coral.qq.com/varticle/"+vid+"/comment/v2?callback=_varticle1005669305commentv2&orinum=10&oriorder=o&pageflag=1&cursor="+cursorid+"&scorecursor=0&orirepnum=2&reporder=o&reppageflag=1&source=9&_=1545445817681"
    data = urllib.request.urlopen(url).read().decode("utf-8")
    return data
#分别构建筛选id、用户名、评论内容等信息的正则表达式
idpat = '"targetid":"(.*?)"' #python中单引号和双引号
userpat = '"userid":"(.*?)",'
conpat = '"content":"(.*?)",'
cursoridpat = '"last":"(.*?)",'
#第一层循环，代表爬取多少页，每一次外层循环爬取一页
for i in range(1,4):
    print("---------------------------")
    print("第"+str(i)+"页评论内容")
    data = craw(vid,cursorid)
    idlist = re.compile(idpat,re.S).findall(data)
    userlist = re.compile(userpat,re.S).findall(data)
    conlist = re.compile(conpat,re.S).findall(data)
    cursorid = re.compile(cursoridpat,re.S).findall(data)[0]
    #第二层循环，根据爬取的结果提取并处理每条评论的信息，一页10条评论
    for j in range(0,10):
        try:
            print("用户名是："+eval('u"'+userlist[j]+'"'))
            print("评论内容是："+eval('u"'+conlist[j]+'"'))
        except Exception as e:
            print(str(e))
        
        print("\n")
    #将comid改变为该页的最后一条评论id，实现不断自动加载
    #print("！！！！长度为"+str(len(idlist))
    #cursorid=idlist[9]

#分别根据正则表达式查找所有id、用户名、评论内容等信息
idlist = re.compile(idpat,re.S).findall(data)
userlist = re.compile(userpat,re.S).findall(data)
conlist = re.compile(conpat,re.S).findall(data)
#通过循环将获取到的各信息遍历出来
for i in range(0,10):
    #输出对应信息，并对字符串进行unicode编码，正常显示
    print("用户名是："+eval('u"'+userlist[i]+'"'))
    print("评论内容是："+eval('u"'+conlist[i]+'"'))
    print("\n")

---------------------------
第1页评论内容
用户名是：234781542
评论内容是：蓝洁瑛，一路走好。愿天堂没有痛苦。


用户名是：102140877
评论内容是：大话西游要重映，刚刚重温一遍。。。割了吧。。。十多年前看不懂，是笑着看完，十多年后看懂了，哭着看完。有多少男孩像条狗一样，没有珍惜曾经的那份感情，现如今却已成为难以抹灭的伤痕。总会有一个女孩出现，让男孩变成男人，再见青春，再见我的爱人，如果上天再给我一次机会，我会紧抱着你对你说，我爱你，永远都不分开



用户名是：12544640
评论内容是：别人笑我太疯癫，我笑他人看不穿。星爷电影表达的意思不到一定年纪是不会懂的，很多台词都很多深意，(那个人好像一条狗)，这句话十几年后我才明白。表达的意思太超前，当年大部分人都看不懂，大话西游这个经典无法超越，连很多台词至今都无法难忘。星爷是我最佩服的一个电影人。看了他很多电影，却没买过一张电影票，我欠星爷一张票。


用户名是：179528875
评论内容是：说真的这么多年了，第一次认真看完这部电影。


用户名是：117647702
评论内容是：紫霞到底爱的是那个放荡不羁的至尊宝，还是脚踏七彩祥云，身披金甲圣衣，法力无边的齐天大圣？


用户名是：236684428
评论内容是：看完前任三又来看这个的举手


用户名是：32502918
评论内容是：如今我们再次观看大话西游，我们谁都想找一个紫霞仙子，也想做一个至尊宝，但是最后到头来我们都是那城墙下面的看客，一个个回忆着自己青涩懵懂的青春年华，不经意间竟已到了迟暮之年


用户名是：407405399
评论内容是：我也是因为惦念蓝洁瑛才认真看完此电影，人漂亮，演技也好，就是命不好


用户名是：76763831
评论内容是：这演得怪怪的，看不懂！演的是啥呀


用户名是：20128166
评论内容是：感情真的好细腻，可惜了朱茵和星星的爱情


---------------------------
第2页评论内容
用户名是：25586719
评论内容是：每一遍看都会哭一遍 真的好感人


用户名是：183227818
评论内容是：星爷用一生回应了那一句“他很花心”，如今满头花白孑然一人。用他的智慧与青春年华，给世人留下了无数欢声笑语和人生感悟！“从前直到现在，爱还在”《西游·降魔篇》

## 小结

1）爬虫的定向爬取技术就是根据设置的主体，对要爬取的网址或者网页中的内容进行筛选。  
2）互联网信息海量，为高效，应根据设定的主体，拟定出对应的爬取策略与爬取规则。  
3）python中，进行信息筛选的方法策略主要有：正则表达式、Xpath表达式、通过xslt筛选。

# 第10章 了解爬虫框架

## 什么是python爬虫框架

爬虫项目的半成品，将常见爬虫功能的实现代码写好（将一些常见的功能代码、业务逻辑等进行封装，从而能够让我们更高效），留下一些接口，后续根据实际情况，编写少量需要变动的代码部分，按照需求调用接口，即可实现一个爬虫项目。

## 常见的爬虫框架

大部分框架大同小异，只需深入了解一种接口。我们以python爬虫框架中的Scrapy框架作为重点讲解内容。  
python常见框架：  
1）Scrapy框架  
2）Crawley框架  
3）Portia框架  
4）newspaper框架  
5）python-goose框架

## 认识Scrapy框架

应用领域：网络爬虫开发、数据挖掘、数据监测、自动化测试等。  
官网：http://scrapy.org/  
特征：开源

## 认识Crawley框架

官网：http://project.crawley-cloud.com/  
可在官网对其了解并下载框架。  
主要特点：  
1）高速爬取对应网站内容  
2）可以将爬取到的内容轻松地存储到关系型数据库中，比如MySQL、Oracle、SQLite、Postgres等。  
3）可以将爬取到的数据导出为JSON、XML等格式。  
4）支持非关系型数据库，比如MongoDB、CouchDB等。  
5）支持使用命令行工具。  
6）可以使用你喜欢的工具提取数据，比如使用XPath或者PyQuery等工具。  
7）支持使用Cookie登录并访问那些只有登录才能够访问的网页。  
8）简单易学。

## 认识Portia框架

特征：允许没有任何编程基础的用户可视化地爬取网页的爬虫框架。  
获取code：http://github.com/scrapinghub/portia/  
网页版地址：https://portia.scrapinghub.com  

## 认识newspaper框架

用来提取新闻、文章以及内容分析的爬虫框架，是python的一个库。  
GitHub上的主页地址：https://github.com/codelucas/newspaper  
特点:  
1)比较简洁  
2)速度快  
3)支持多线程  
4)支持十多种语言

## 认识python-goose框架

Goose本来是一款Java写的文章提取工具，有人用python重写了Goose，并将重写后的Goose命名为Python-goose。  
功能：文章提取  
GitHub主页地址：https://github.com/grangier/python-goose  
提取如下信息：  
1）文章主体内容  
2）元描述  
3）元标签  
4）文章中的任何YouTube/vIMEO视频  
5）文章中的主要图片

## 小结

# 第11章 爬虫利器——Scrapy安装与配置

## Windows安装和配置

设置fiddler，取消软件对客户端连接的干扰。

可能需要Visual Studio的支持，建议安装专业版，社区版可能会出现很多问题。

64位版本对应问题：  
import pip._internal  
print(pip._internal.pep425tags.get_supported())  
32位版本对应问题：  
import pip;print(pip.pep425tags.get_supported())

## MAC安装和配置

## Linux安装和配置

## 小结

# 第12章 开启Scrapy爬虫项目之旅

## 认识Scrapy项目的目录结构

## 用Scrapy进行爬虫项目管理

cmd命令中输入：  
scrapy startproject myfirstpjt
**调出startproject的帮助信息：scrapy startproject -h**  
输出：  
**--logfile=FILE          log file. if omitted stderr will be used**  
（指定日志文件file为指定日志文件的路径地址。  命令例子：scrapy startproject --logfile="../logf.log" mypjt1  
**--loglevel=LEVEL, -L LEVEL**  
                        log level (default: DEBUG)  
--loglevel=LEVEL,-L LEVEL参数主要控制日志信息的等级，默认为DEBUG模式，即会将对应调试信息都输出，其它值还有： 
CRITICAL 发生最严重的的错误  
ERROR  发生了必须立即处理的错误  
WARNING 出现一些警告信息，即存在潜在错误  
INFO  输出一些提示信息  
DEBUG 输出一些调试信息，常用于开发阶段  
如果将日志等级设置为DEBUG最低等级，所有调试信息都会输出出来，如果只想输出一些警告以上的日志信息，可将日志等级设置为WARNING.  
设置例子：scrapy startproject --loglevel=DEBUG mypjt2  
--nolog                 disable logging completely  
控制不输出日志信息，如:scrapy startproject --nolog mypjt3  
**--profile=FILE          write python cProfile stats to FILE**  
**--pidfile=FILE          write process ID to FILE**  
**--set=NAME=VALUE, -s NAME=VALUE**  
                        set/override setting (may be repeated)  
--pdb                   enable pdb on failure  
比如，我们希望日志文件存储在当前目录的上一层目录下，并且日志文件名为logf.txt，此时我们可以这样实现：  
scrapy startproject --logfile="../logf.log" mypjt1

## 常用工具命令

两种工具命令：一种为全局命令，一种为项目命令。  
全局命令不需要依靠scrapy项目就可以在全局中直接运行，而项目命令必须要在scrapy项目中才能运行。  

### 全局命令

在项目目录之外，运行 scrapy -h，出现所有的全局命令。  
Available commands:
  bench         Run quick benchmark test（项目命令）  
  check         Check spider contracts  
  crawl         Run a spider  
  edit          Edit spider  
  fetch         Fetch a URL using the Scrapy downloader  
  genspider     Generate new spider using pre-defined templates  
  list          List available spiders  
  parse         Parse URL (using its spider) and print the results  
  runspider     Run a self-contained spider (without creating a project)  
  settings      Get settings values  
  shell         Interactive scraping console  
  startproject  Create new project  
  version       Print Scrapy version  
  view          Open URL in browser, as seen by Scrapy  

**（1）fetch命令**  
在cmd中使用“scrapy fetch 网址”显示爬取对应网址的过程。  
此时，如果在项目目录之外使用该命令，则会调用scrapy默认的爬虫来进行网页爬取，如果在某个项目目录内使用该命令，则会调用该项目中的爬虫来进行网页的爬取。  
在使用fetch命令的时候，同样可以使用某些参数进行相应的控制。  
通过scrapy fetch -h列出所有可以使用的fetch相关参数。  
比如，可以通过--headers参数来控制显示对应的爬虫网页头信息，可通过--nlog参数来控制不显示日志信息，同时，还可以通过--spider=SPIDER参数来控制使用哪个爬虫，通过--logfile=FILE参数来指定存储日志信息的文件，通过--loglevel=LEVEL参数来控制日志等级。  
如：scrapy fetch --headers --nolog http://news.sina.com.cn/

**runspider命令**  
通过runspider命令可以实现不依托scrapy的爬虫项目，直接运行一个爬虫文件。  


In [ ]:
#保存first.py文件
from scrapy.spiders import Spider

class FirstSpider(Spider):
    name = "first"
    allowed_domains = ["baidu.com"]
    start_urls = [
        "http://www.baidu.com"
    ]
    
    def parse(self,response):
        pass

In [ ]:
#输入命令
scrapy runspider --loglevel=INFO first.py

**（3）settings命令**  
如果在scrapy项目目录内使用settings命令，查看的是对应项目的配置信息，如果在scrapy项目目录外使用settings命令，查看的是scrapy默认配置信息。  
myfirstpjt\myfirstpjt文件夹中的settings.py文件，就是该项目的配置信息。  
还可通过命令行查看该项目的配置信息，使用scrapy settings --get BOT_NAME查看配置信息中BOT_NAME对应的值。  
可退出项目目录查看默scrapy认配置信息。

**(4)shell命令**  

通过shell命令可以启动scrapy的交互终端（scrapy shell）。  
scrapy的交互终端经常在开发及调试的时候用到，使用scrapy的交互终端可以实现在不启动scrapy爬虫的情况下，对网站相应进行调试，在交互终端中，也可以写一些python代码进行相应测试。  
比如，使用shell命令为爬取百度首页创建一个交互终端环境，并设置为不输出日志信息：  
**scrapy shell http://www.baidu.com --nolog**  
执行该命令之后，会出现可以使用的scrapy对象及快捷命令，比如item、response、settings、spider等。进行交互模式后，在“>>>”后可以输入交互命令及相应的代码。  
通过XPath表达式进行提取网页标题。“/html/head/title"的意思是：提取该网页\<html>标签下的\<head>标签中的\<title>标签对应的信息。我们知道，此时该标签中的信息即为该网页的标题信息。  
**>>> ti=sel.xpath("/html/head/title")**
**>>> print(ti)**

**(5)startproject命令**

**（6）version命令**

In [ ]:
通过version，可以直接显示scrapy的版本相关信息。  
代码实现： scrapy version  
输出：1.5.1  
查看其它版本信息：scrapy version -v  
输出：  
Scrapy       : 1.5.1  
lxml         : 4.1.0.0  
libxml2      : 2.9.5
cssselect    : 1.0.3
parsel       : 1.5.1
w3lib        : 1.19.0
Twisted      : 18.9.0
Python       : 3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]
pyOpenSSL    : 17.2.0 (OpenSSL 1.0.2n  7 Dec 2017)
cryptography : 2.0.3
Platform     : Windows-10-10.0.14393-SP0

**（7）view命令**  
通过view命令，我们可以实现下载某个网页并用浏览器查看的功能。  
比如，网易：scrapy view http://news.163.com/  
执行该命令之后，会自动在浏览器中打开下载的该网页，地址为本地？

### 项目命令

进入项目，scrapy -h  
Available commands:
  bench         Run quick benchmark test
  check         Check spider contracts
  crawl         Run a spider
  edit          Edit spider
  fetch         Fetch a URL using the Scrapy downloader
  genspider     Generate new spider using pre-defined templates
  list          List available spiders
  parse         Parse URL (using its spider) and print the results
  runspider     Run a self-contained spider (without creating a project)
  settings      Get settings values
  shell         Interactive scraping console
  startproject  Create new project
  version       Print Scrapy version
  view          Open URL in browser, as seen by Scrapy
**除去全局命令之后，剩下的即为项目命令**  
主要命令有：bench\check\crawl\edit\genspider\list\parse  
**值得注意的是**  
全局命令既可以在非scrapy项目文件夹中使用，也可以在项目文件夹中使用；而项目命令一般只能在scrapy项目文件夹中用。

**（1）bench命令**  
使用此命令可以测试本地硬件的性能。
命令：scrapy bench   
根据输出结果可查看硬件性能，每分钟大概爬取多少个网页，但是实际情况不定。

**（2）Genspider命令**  
可以使用genspider命令来创建scrapy爬虫文件，快速创建爬虫文件的方式。  
使用该命令可以基于现有的爬虫模板直接生成一个新的爬虫文件，非常方便。  
例子：scrapy genspider -l  
可以看到当前可以使用的模板有：  
Available templates:
  basic
  crawl
  csvfeed
  xmlfeed
例子：scrapy genspider -t basic weisuen iqianyue.com  
使用basic模板创建新的爬虫文件weisuen.py。  
可以通过-d参数查看某个爬虫模板的内容，例如： scrapy genspider -d csvfeed

**(3)check命令**  
爬虫测试比较麻烦，所以在scrapy中使用合同/契约（contract）的方式对爬虫进行测试。  
例子：scrapy check weisuen (爬虫名，而不是爬虫文件名）  

**(4)Crawl命令**  
可以通过crawl命令来启动某个爬虫，启动格式是“scrapy crawl爬虫名”  
例子：scrapy crawl weisuen --loglevel=INFO  

**(5)List命令**  
通过scrapy中的list命令，可以列出当前可使用的爬虫文件  
例如：进入myfirstpjt项目所在目录：scrapy list，可看到当前可使用的爬虫文件：weisuen

**(6)Edit命令**  
通过scrapy中的edit命令，可以直接打开对应编辑器对爬虫文件对爬虫文件进行编辑，该命令在Windows中执行会出现一点问题，一般会用python IDE（如PyCharm）直接对爬虫项目进行管理和编辑，该命令在Linux中很方便。   
例如：scrapy list（查看文件）  
scrapy edit abc（编辑文件abc）  

**（7）Parse命令**  
实现获取指定URL网址，并使用对应的爬虫文件进行处理和分析。  
比如，scrapy parse http://www.baidu.com --nolog 获取百度首页，这里没有指定爬虫文件和函数，所以会用默认的。  
查看scrapy parse的参数： d:\python\crawler\myfirstpjt>scrapy parse -h  
Options
_____________________________________________________________________________
--help, -h              show this help message and exit
--spider=SPIDER         use this spider without looking for one #指定爬虫文件
-a NAME=VALUE           set spider argument (may be repeated) #设置spider参数
--pipelines             process items through pipelines #处理items
--nolinks               don't show links to follow (extracted requests) #不展示提取到的链接信息
--noitems               don't show scraped items #不展示得到的items
--nocolour              avoid using pygments to colorize the output #输出结果颜色不高亮
--rules, -r             use CrawlSpider rules to discover the callback 
                        #使用CrawlSpider规则处理回调函数
--callback=CALLBACK, -c CALLBACK
                        use this callback for parsing, instead looking for a
                        callback #使用spider中用于处理返回的响应的回调函数
--meta=META, -m META    inject extra meta into the Request, it must be a valid
                        raw json string
--depth=DEPTH, -d DEPTH #设置爬行深度，默认深度为1
                        maximum depth for parsing requests [default: 1]
--verbose, -v           print each depth level one by one #显示每层的详细信息

Global Options
__________________________________________________________________
--logfile=FILE          log file. if omitted stderr will be used
--loglevel=LEVEL, -L LEVEL
                        log level (default: DEBUG)
--nolog                 disable logging completely
--profile=FILE          write python cProfile stats to FILE
--pidfile=FILE          write process ID to FILE
--set=NAME=VALUE, -s NAME=VALUE
                        set/override setting (may be repeated)
--pdb                   enable pdb on failure

## 实战：Items的编写

In [2]:
#理解Items
import scrapy
class person(scrapy.Item):
    name = scrapy.Field()
    job = scrapy.Field()
    email = scrapy.Field()
yang = person(name = "yang",job="student",email="17888828903@163.com")
print(yang)

{'email': '17888828903@163.com', 'job': 'student', 'name': 'yang'}


In [4]:
yang["job"]

'student'

In [6]:
yang['name']='ye'
yang['name']

'ye'

In [7]:
#获取字段名
yang.keys()

dict_keys(['name', 'job', 'email'])

In [8]:
#获取该对象中的项目视图（ItemView)
yang.items()

ItemsView({'email': '17888828903@163.com', 'job': 'student', 'name': 'ye'})

## 实战：Spider的编写

Spider类是Scrapy中一个与爬虫相关的基类，所有爬虫文件必须继承该类（scrapy.Spider）。  
爬虫文件：定义如何对网站进行相应的爬取，爬取动作及数据提取等操作都在此定义和编写。  
例如：genspider命令创建一个爬虫文件，打开、修改和编写。  

##  Xpath基础

手写网络爬虫的时候，使用正则表达式对爬取到的数据进行筛选和提取，在scrapy会使用XPath表达式进行数据筛选和提取。  
XPath是一种XML路径语言，通过该语言可以在XML文档中迅速地查找相应信息，XPath通常叫做XPath selector。   
**(1)**在XPath表达式中，使用“/”可以选择某个标签，并且可以使用哦“/”进行多层标签的查找。  
比如在html文件中查找，“/html/body/h2”查找标签/*<h2></h2>*/对应的内容。如果要获取该标签中的文本信息，通过text()实现。/html/body/h2/text()  
**(2)**使用“//”可以提取某个标签的所有信息。比如上方代码中出现了多个<p>标签，如果要将所有<p>标签的所有信息都提取出来，可以使用“//”实现。如：//p。  
**(3)**如果要获取所有属性X的值为Y的<z>标签的内容，可以通过//z[@X="Y"]获取。  
比如：//img[@class="f1"]  
**(4)**更多XPath的学习查看官方手册。

## Spider类参数传递